In [1]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import sys
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from torch.utils.data import Dataset
import scipy.io as io
import random
from datetime import datetime

# 1.数据集定义


In [2]:
from torch.utils.data import Dataset

class MyData(Dataset):
    def __init__(self, pics, labels):
        self.pics = pics
        self.labels = labels
 
    def __getitem__(self, index):
        assert index < len(self.pics)
        return torch.Tensor([self.pics[index]]), self.labels[index]
 
    def __len__(self):
        return len(self.pics)
 
    def get_tensors(self):
        return torch.Tensor([self.pics]), torch.Tensor(self.labels)

'''
for x, y in trainloader:
  print(x.shape)
  print(y.shape)

# torch.Size([4, 1, 64, 64])
# torch.Size([4])
'''

'\nfor x, y in trainloader:\n  print(x.shape)\n  print(y.shape)\n\n# torch.Size([4, 1, 64, 64])\n# torch.Size([4])\n'

In [6]:
train_pics_dict = np.load("/content/drive/MyDrive/Colab_Notebooks/CWRU/data/train_pics.npz")
train_labels_dict = np.load("/content/drive/MyDrive/Colab_Notebooks/CWRU/data/train_labels.npz")
 
train_pics = []
train_labels = []
 
for i in train_pics_dict.files:
    train_pics.append(train_pics_dict[i])
    train_labels.append(int(train_labels_dict[i]))

# 2.网络构建

In [3]:
class CNN_Net(nn.Module):
    def __init__(self):
        super(CNN_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, padding=2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(4 * 4 * 256, 2560)
        self.fc2 = nn.Linear(2560, 10)
        # self.fc2 = nn.Linear(2560, 2)
        # self.fc2 = nn.Linear(2560, 4)
 
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = x.view(-1, 4 * 4 * 256)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
net = CNN_Net()
print(net)

CNN_Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4096, out_features=2560, bias=True)
  (fc2): Linear(in_features=2560, out_features=10, bias=True)
)


# 3.模型训练

In [4]:
from torch import optim
import torch.nn as nn
import torch

def train_net(net, train_iter, device, num_epochs, optimizer, batch_size, lr):
  net = net.to(device)
  print("training on ", device)
  # 定义Loss
  criterion = nn.CrossEntropyLoss()

  # best_loss
  best_loss = float('inf')

  # train
  for epoch in range(num_epochs):
    print("epoch;%d"%epoch)
    # 按batch_size开始训练
    for image, label in train_iter:
        # 将数据拷贝到device
        image = image.to(device)
        label = label.to(device)

        # 计算loss
        loss = criterion(net(image), label)
        print('Loss/train', loss.item())

        # 保存loss值最小的网络参数
        if loss < best_loss:
            best_loss = loss
            torch.save(net.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/CWRU/cnn_best_model.pth')

        # 参数更新
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [7]:
# train_net(net, device, num_epochs, optimizer, batch_size, lr)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
 

net = CNN_Net()
batch_size = 500
num_epochs, lr = 50, 0.001
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)

# 加载训练集
train_data = MyData(train_pics, train_labels)
# print(len(train_pics))  20000

train_iter = torch.utils.data.DataLoader(dataset=train_data,
                        batch_size=batch_size,
                        shuffle=True,
                        num_workers=4)  

train_net(net, train_iter, device, num_epochs, optimizer, batch_size, lr)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


training on  cuda
epoch;0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting 

Loss/train 2.300976276397705
Loss/train 2.301093339920044
Loss/train 2.301025152206421
Loss/train 2.3027193546295166
Loss/train 2.3020806312561035
Loss/train 2.300412893295288
Loss/train 2.2994136810302734
Loss/train 2.299929618835449
Loss/train 2.300812005996704
Loss/train 2.299257278442383
Loss/train 2.2983016967773438
Loss/train 2.2978639602661133
Loss/train 2.2977347373962402
Loss/train 2.2958316802978516
Loss/train 2.2949516773223877
Loss/train 2.2955281734466553
Loss/train 2.29464054107666
Loss/train 2.293168067932129
Loss/train 2.293980598449707
Loss/train 2.292896032333374
Loss/train 2.2926416397094727
Loss/train 2.2913074493408203
Loss/train 2.2884345054626465
Loss/train 2.2893388271331787
Loss/train 2.2906880378723145
Loss/train 2.2880823612213135
Loss/train 2.2867448329925537
Loss/train 2.290087938308716
Loss/train 2.2873408794403076
Loss/train 2.2863929271698
Loss/train 2.283555269241333
Loss/train 2.2801461219787598
Loss/train 2.2831380367279053
Loss/train 2.28213858604431

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting 

...

Loss/train 0.12189846485853195
Loss/train 0.13389500975608826
Loss/train 0.1338745653629303
Loss/train 0.15307995676994324
Loss/train 0.16977636516094208
Loss/train 0.11081646382808685
Loss/train 0.09244950115680695
Loss/train 0.13950884342193604
Loss/train 0.13923725485801697
Loss/train 0.1476946473121643
Loss/train 0.1416531503200531
Loss/train 0.14505061507225037
Loss/train 0.1240745410323143
Loss/train 0.12193826586008072
Loss/train 0.15171211957931519
Loss/train 0.1388826072216034
Loss/train 0.11414604634046555
Loss/train 0.11686820536851883
Loss/train 0.14744146168231964
Loss/train 0.11870965361595154
Loss/train 0.13870006799697876
Loss/train 0.1509736031293869
Loss/train 0.1441003978252411
Loss/train 0.12815792858600616
Loss/train 0.14907069504261017
Loss/train 0.12265217304229736
Loss/train 0.1353832632303238
Loss/train 0.13508203625679016
Loss/train 0.17432068288326263
Loss/train 0.13593067228794098
Loss/train 0.12448783963918686
Loss/train 0.13297154009342194
Loss/train 0.132

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting 

Loss/train 0.0989653468132019
Loss/train 0.16538271307945251
Loss/train 0.14386257529258728
Loss/train 0.12801493704319
Loss/train 0.1668105572462082
Loss/train 0.14731143414974213
Loss/train 0.13187962770462036
Loss/train 0.09937756508588791
Loss/train 0.1349603682756424
Loss/train 0.1574619561433792
Loss/train 0.1157754510641098
Loss/train 0.13661746680736542
Loss/train 0.09966283291578293
Loss/train 0.13190321624279022
Loss/train 0.14632059633731842
Loss/train 0.1173735037446022
Loss/train 0.11733534187078476
Loss/train 0.10994618386030197
Loss/train 0.12480152398347855
Loss/train 0.12000180780887604
Loss/train 0.11111568659543991
Loss/train 0.12332358956336975
Loss/train 0.09805652499198914
Loss/train 0.08902765810489655
Loss/train 0.11926291882991791
Loss/train 0.15019580721855164
Loss/train 0.1349080502986908
Loss/train 0.1270669400691986
Loss/train 0.10311912000179291
Loss/train 0.14722460508346558
Loss/train 0.09538708627223969
Loss/train 0.13930045068264008
Loss/train 0.086161

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting 

Loss/train 0.10628978908061981
Loss/train 0.1244175136089325
Loss/train 0.13662128150463104
Loss/train 0.1218656674027443
Loss/train 0.1094876229763031
Loss/train 0.11772654950618744
Loss/train 0.09583523869514465
Loss/train 0.11322851479053497
Loss/train 0.13050708174705505
Loss/train 0.10923848301172256
Loss/train 0.12021493166685104
Loss/train 0.09293469041585922
Loss/train 0.11168309301137924
Loss/train 0.12933363020420074
Loss/train 0.1389605700969696
Loss/train 0.11770567297935486
Loss/train 0.12771590054035187
Loss/train 0.1182728037238121
Loss/train 0.11469132453203201
Loss/train 0.13146010041236877
Loss/train 0.10582908987998962
Loss/train 0.14033229649066925
Loss/train 0.1139586940407753
Loss/train 0.12250712513923645
Loss/train 0.14417612552642822
Loss/train 0.10309658944606781
Loss/train 0.1274895817041397
Loss/train 0.11799976974725723
Loss/train 0.10015474259853363
Loss/train 0.10250783711671829
Loss/train 0.12713521718978882
Loss/train 0.0957188755273819
Loss/train 0.098

# 4.模型测试

In [8]:
# 加载测试数据库
test_pics_dict = np.load("/content/drive/MyDrive/Colab_Notebooks/CWRU/data/test_pics.npz")
test_labels_dict = np.load("/content/drive/MyDrive/Colab_Notebooks/CWRU/data/test_labels.npz")

test_pics = []
test_labels = []

for i in test_pics_dict.files:
    test_pics.append(test_pics_dict[i])
    test_labels.append(int(test_labels_dict[i]))

test_data = MyData(test_pics, test_labels)
test_iter = torch.utils.data.DataLoader(dataset=test_data,
                       batch_size=batch_size,
                       shuffle=True,
                       num_workers=4) 

PATH = "/content/drive/MyDrive/Colab_Notebooks/CWRU/cnn_best_model.pth"
net = CNN_Net().to(device)
net.load_state_dict(torch.load(PATH))
print("load success")

correct = 0
total =0
with torch.no_grad():
    for image, label in test_iter:
        image = image.to(device)
        label = label.to(device)
        output = net(image)
        _, pred = torch.max(output, 1)
        total += batch_size
        correct += (pred == label).sum().item()
print("Correct=%d"%(correct / total *100))

load success


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider conve

Correct=95
